# Power Coder

1. Convert code between two programming language; supporting languages are Python, Java, JavaScript, TypeScript, C, C++, C#, Go, Rust, Kotlin, Swift, PHP, Julia
2. Automatically add docstring/comments based on selected comment style
3. Automatically generate unit tests based on selected unit test style
4. Supporting models: gpt-4o, claude-3-5-sonnet-20240620, gemini-2.5-flash"

In [ ]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
gemini_via_openai_client = OpenAI(
    api_key=os.environ['GOOGLE_API_KEY'], 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-2.5-flash"

## 1. Convesion Part

In [ ]:
def convert_system_prompt_for(in_lang, out_lang):
    convert_system_message = f"You are an assistant that reimplements {in_lang} code in high performance {out_lang}. "
    convert_system_message += f"Respond only with {out_lang} code; use comments sparingly and do not provide any explanation other than occasional comments. "
    convert_system_message += f"The {out_lang} response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."
    return convert_system_message

In [ ]:
def convert_user_prompt_for(in_lang, out_lang, input_instruct, in_code):
    convert_user_prompt = f"Rewrite this {in_lang} code in {out_lang} with the fastest possible implementation that produces identical output in the least time. "
    convert_user_prompt += f"Respond only with {out_lang} code; do not explain your work other than a few comments. "
    convert_user_prompt += f"Pay attention to number types to ensure no int overflows. Remember to include all necessary {out_lang} packages or modules, for example, iomanip for C++.\n\n"
    if input_instruct:
        convert_user_prompt += "Addtional instruction is: " + input_instruct
    convert_user_prompt += in_code
    return convert_user_prompt

In [ ]:
def convert_messages_for(in_lang, out_lang, input_instruct, in_code):
    return [
        {"role": "system", "content": convert_system_prompt_for(in_lang, out_lang)},
        {"role": "user", "content": convert_user_prompt_for(in_lang, out_lang, input_instruct, in_code)}
    ]

In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
def convert_stream_gpt(in_lang, out_lang, input_instruct, in_code):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=convert_messages_for(in_lang, out_lang, input_instruct, in_code), temperature=0.0, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def convert_stream_claude(in_lang, out_lang, input_instruct, in_code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        temperature=0.0,
        system=convert_system_prompt_for(in_lang, out_lang),
        messages=[{"role": "user", "content": convert_user_prompt_for(in_lang, out_lang, input_instruct, in_code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply

In [ ]:
def convert_stream_gemini(in_lang, out_lang, input_instruct, in_code):    
    stream = gemini_via_openai_client.chat.completions.create(model=GEMINI_MODEL, messages=convert_messages_for(in_lang, out_lang, input_instruct, in_code), temperature=0.0, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def optimize(in_lang, out_lang, in_code, input_instruct, convert_model):
    if "gpt" in convert_model.lower():
        result = convert_stream_gpt(in_lang, out_lang, input_instruct, in_code)
    elif "claude" in convert_model.lower():
        result = convert_stream_claude(in_lang, out_lang, input_instruct, in_code)
    elif "gemini" in convert_model.lower():
        result = convert_stream_gemini(in_lang, out_lang, input_instruct, in_code)
    else:
        raise ValueError("Unknown convert model")
    for stream_so_far in result:
        yield stream_so_far        

## 2. Comment part

In [ ]:
def comment_system_prompt_for(lang, comment_style):
    comment_system_message = f"You are an assistant that generate necessary, concise and clear comment/docstring for the {lang} code by applying {comment_style} comment style. "
    comment_system_message += f"Respond only with added comments, and do not provide any redundant explanation. "
    return comment_system_message

In [ ]:
def comment_user_prompt_for(lang, code, comment_style):
    comment_user_prompt = f"Add the comments/docstring on the given code for the {lang} programming language in {comment_style} comment style. "
    comment_user_prompt += f"Respond only with added comments, and do not provide any redundant explanation.\n\n"
    comment_user_prompt += f"The given code is as follows: "
    comment_user_prompt += code
    return comment_user_prompt

In [ ]:
def comment_messages_for(lang, code, comment_style):
    return [
        {"role": "system", "content": comment_system_prompt_for(lang, comment_style)},
        {"role": "user", "content": comment_user_prompt_for(lang, code, comment_style)}
    ]

In [ ]:
def comment_stream_gpt(lang, code, comment_style):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=comment_messages_for(lang, code, comment_style), temperature=0.0, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def comment_stream_claude(lang, code, comment_style):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        temperature=0.0,
        system=comment_system_prompt_for(lang, comment_style),
        messages=[{"role": "user", "content": comment_user_prompt_for(lang, code, comment_style)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply

In [ ]:
def comment_stream_gemini(lang, code, comment_style):    
    stream = gemini_via_openai_client.chat.completions.create(model=GEMINI_MODEL, messages=comment_messages_for(lang, code, comment_style), temperature=0.0, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def generate_comments_via_model(lang, code, comment_style, comment_model):
    if "gpt" in comment_model.lower():
        result = comment_stream_gpt(lang, code, comment_style)
    elif "claude" in comment_model.lower():
        result = comment_stream_claude(lang, code, comment_style)
    elif "gemini" in comment_model.lower():
        result = comment_stream_gemini(lang, code, comment_style)
    else:
        raise ValueError("Unknown comment model")
    for stream_so_far in result:
        yield stream_so_far        

In [ ]:
def generate_comments_fn(comment_option, in_lang, out_lang, in_code, out_code, in_comment_style, out_comment_style, comment_model):
    if 'input' in comment_option:
        in_gen = generate_comments_via_model(in_lang, in_code, in_comment_style, comment_model)
        for in_output in in_gen:
            yield in_output, ""
    elif 'output' in comment_option:
        out_gen = generate_comments_via_model(out_lang, out_code, out_comment_style, comment_model)
        for out_output in out_gen:
            yield "", out_output
    elif 'both' in comment_option:
        in_gen = generate_comments_via_model(in_lang, in_code, in_comment_style, comment_model)
        out_gen = generate_comments_via_model(out_lang, out_code, out_comment_style, comment_model)
        for in_output, out_output in zip(in_gen, out_gen):
            yield in_output, out_output

## 3. Unit test part

In [ ]:
def unit_test_system_prompt_for(lang, unit_test_style):
    unit_test_system_message = f"You are an assistant that generate necessary, concise, clear and executable unit tests for the {lang} code by applying {unit_test_style} unit test style. "
    unit_test_system_message += f"Respond only with generated unit tests; use comments sparingly and do not provide any explanation other than occasional comments. "
    return unit_test_system_message

In [ ]:
def unit_test_user_prompt_for(lang, code, unit_test_style):
    unit_test_user_prompt = f"Add the unit tests on the given code for the {lang} programming language in {unit_test_style} unit test style. "
    unit_test_user_prompt += f"Respond only with generated unit tests; use comments sparingly and do not provide any explanation other than occasional comments.\n\n"
    unit_test_user_prompt += f"The given code is as follows: "
    unit_test_user_prompt += code
    return unit_test_user_prompt

In [ ]:
def unit_test_messages_for(lang, code, unit_test_style):
    return [
        {"role": "system", "content": unit_test_system_prompt_for(lang, unit_test_style)},
        {"role": "user", "content": unit_test_user_prompt_for(lang, code, unit_test_style)}
    ]

In [ ]:
def unit_test_stream_gpt(lang, code, unit_test_style):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=unit_test_messages_for(lang, code, unit_test_style), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def unit_test_stream_claude(lang, code, unit_test_style):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=unit_test_system_prompt_for(lang, unit_test_style),
        messages=[{"role": "user", "content": unit_test_user_prompt_for(lang, code, unit_test_style)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply

In [ ]:
def unit_test_stream_gemini(lang, code, unit_test_style):    
    stream = gemini_via_openai_client.chat.completions.create(model=GEMINI_MODEL, messages=unit_test_messages_for(lang, code, unit_test_style), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def generate_unit_test_via_model(lang, code, unit_test_style, unit_test_model):
    if "gpt" in unit_test_model.lower():
        result = unit_test_stream_gpt(lang, code, unit_test_style)
    elif "claude" in unit_test_model.lower():
        result = unit_test_stream_claude(lang, code, unit_test_style)
    elif "gemini" in unit_test_model.lower():
        result = unit_test_stream_gemini(lang, code, unit_test_style)
    else:
        raise ValueError("Unknown unit test model")
    for stream_so_far in result:
        yield stream_so_far        

In [ ]:
def generate_unit_test_fn(unit_test_option, in_lang, out_lang, in_code, out_code, in_unit_test_style, out_unit_test_style, unit_test_model):
    if 'input' in unit_test_option:
        in_gen = generate_unit_test_via_model(in_lang, in_code, in_unit_test_style, unit_test_model)
        for in_output in in_gen:
            yield in_output, ""
    elif 'output' in unit_test_option:
        out_gen = generate_unit_test_via_model(out_lang, out_code, out_unit_test_style, unit_test_model)
        for out_output in out_gen:
            yield "", out_output
    elif 'both' in unit_test_option:
        in_gen = generate_unit_test_via_model(in_lang, in_code, in_unit_test_style, unit_test_model)
        out_gen = generate_unit_test_via_model(out_lang, out_code, out_unit_test_style, unit_test_model)
        for in_output, out_output in zip(in_gen, out_gen):
            yield in_output, out_output

## 4. Gradio UI part

In [ ]:
LANGUAGE_INFO = {
    "Python": {
        "doc_style": ["Google-style", "NumPy-style", "reST", "Doxygen"],
        "unit_test_style": ["unittest", "pytest", "doctest"]
    },
    "Java": {
        "doc_style": ["Javadoc"],
        "unit_test_style": ["JUnit4", "JUnit5", "TestNG"]
    },
    "JavaScript": {
        "doc_style": ["JSDoc"],
        "unit_test_style": ["Jest", "Mocha + Chai", "Jasmine"]
    },
    "TypeScript": {
        "doc_style": ["JSDoc", "TSDoc"],
        "unit_test_style": ["Jest", "Mocha + Chai", "Vitest"]
    },
    "C": {
        "doc_style": ["Doxygen"],
        "unit_test_style": ["Google Test (gtest)", "CppUnit", "Catch2"]
    },
    "C++": {
        "doc_style": ["Doxygen"],
        "unit_test_style": ["Google Test (gtest)", "CppUnit", "Catch2"]
    },
    "C#": {
        "doc_style": ["XML comments"],
        "unit_test_style": ["xUnit", "NUnit", "MSTest"]
    },
    "Go": {
        "doc_style": ["Godoc"],
        "unit_test_style": ["Built-in testing package"]
    },
    "Rust": {
        "doc_style": ["Rustdoc", "Markdown"],
        "unit_test_style": ["Built-in #[test] annotation"]
    },
    "Kotlin": {
        "doc_style": ["KDoc"],
        "unit_test_style": ["JUnit", "Kotest", "Spek"]
    },
    "Swift": {
        "doc_style": ["Mark-style comments"],
        "unit_test_style": ["XCTest"]
    },
    "PHP": {
        "doc_style": ["PHPDoc"],
        "unit_test_style": ["PHPUnit"]
    },
    "Julia": {
        "doc_style": ["Markdown"],
        "unit_test_style": ["Built-in Test standard library"]
    }
}
LANGUAGES = list(LANGUAGE_INFO.keys())

In [ ]:
with gr.Blocks(title="Power Coder", theme=gr.themes.Citrus(), css="""
.selected {
    background-color: orange !important;
    box-shadow: 0 4px 12px rgba(255, 140, 0, 0.5) !important;
    color: black;
}
.unselected {
    background-color: gray !important;
    box-shadow: 0 4px 12px rgba(128, 128, 128, 0.4);
    color: white;
}
""") as ui:
    current_selected = gr.State("")
    initial_in_lang = "Python"
    initial_out_lang = "Java"
    in_comment_style_choices = ["Standard"] + LANGUAGE_INFO[initial_in_lang]["doc_style"]
    out_comment_style_choices = ["Standard"] + LANGUAGE_INFO[initial_out_lang]["doc_style"]
    in_unit_test_style_choices = ["Standard"] + LANGUAGE_INFO[initial_in_lang]["unit_test_style"]
    out_unit_test_style_choices = ["Standard"] + LANGUAGE_INFO[initial_out_lang]["unit_test_style"]
    in_code_file_name = gr.State("in_code.txt")
    out_code_file_name = gr.State("out_code.txt")
    in_comments_file_name = gr.State("in_comments.txt")
    out_comments_file_name = gr.State("out_comments.txt")
    in_unit_test_file_name = gr.State("in_unit_tests.txt")
    out_unit_test_file_name = gr.State("out_unit_tests.txt")
    
    
    gr.Markdown("## Code Helper")

    def load_file_content(file):
        if file is None:
            return ""
        with open(file.name, "r", encoding="utf-8") as f:
            return f.read()

    def change_lang(lang):
        comment_style_choices = ["Standard"] + LANGUAGE_INFO[lang]["doc_style"]
        unit_test_style_choices = ["Standard"] + LANGUAGE_INFO[lang]["unit_test_style"]
        return (
            gr.update(choices=comment_style_choices, value=str(comment_style_choices[0])), 
            gr.update(choices=unit_test_style_choices, value=str(unit_test_style_choices[0]))
        )

    def download_fn(in_text, out_text, in_file_name, out_file_name):
        if in_text:
            with open(in_file_name, "w") as f:
                f.write(in_text)
        if out_text:
            with open(out_file_name, "w") as f:
                f.write(out_text)
        
    # Conversion part
    with gr.Row():
        in_lang = gr.Dropdown(choices=LANGUAGES, label="Select input language", value=initial_in_lang, interactive=True)
        out_lang = gr.Dropdown(choices=LANGUAGES, label="Select output language", value=initial_out_lang, interactive=True)
    with gr.Row():
        input_file = gr.File(label="Upload a source code file or input below")
        input_instruct = gr.Textbox(
            label="Additional instruction(optional)",
            placeholder="Enter the instruction you want the ouput code to follow...\n\nFor example: Define the variable using snake_case style.",
            lines=8
        )
    with gr.Row():
        in_code = gr.Textbox(label="Input Code:", value=python_hard, lines=10)
        out_code = gr.Textbox(label="Output Code:", lines=10)
    with gr.Row():
        convert_model = gr.Dropdown(["Claude", "GPT", "Gemini"], label="Select model", value="Claude")
    with gr.Row():
        convert = gr.Button("Convert code")
        download_code = gr.Button("Download code")

    gr.HTML("<hr style='border: none; height: 1px; background-color: #333;'>")

    def show_comment(current_selected):
        if current_selected == "comment":
            return (
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(elem_classes=["unselected"]),
                gr.update(elem_classes=["unselected"]),
                ""
            )
        else:
            return (
                gr.update(visible=True),
                gr.update(visible=False),
                gr.update(elem_classes=["selected"]),
                gr.update(elem_classes=["unselected"]),
                "comment"
            )

    def show_unit_test(current_selected):
        if current_selected == "unit_test":
            return (
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(elem_classes=["unselected"]),
                gr.update(elem_classes=["unselected"]),
                ""
            )
        else:
            return (
                gr.update(visible=False),
                gr.update(visible=True),
                gr.update(elem_classes=["unselected"]),
                gr.update(elem_classes=["selected"]),
                "unit_test"
            )
    
    with gr.Blocks() as demo:
        with gr.Row():
            comment_show_up = gr.Button("Comment", elem_id="comment-btn", elem_classes=["unselected"])
            unit_test_show_up = gr.Button("Unit Test",  elem_id="unit-test-btn", elem_classes=["unselected"])
        
        comment_section = gr.Column(visible=False)
        unit_test_section = gr.Column(visible=False)
        
    with comment_section:
        # Comment section
        with gr.Row():
            comment_option = gr.Radio(
                choices=[
                    "Comment input code",
                    "Comment output code",
                    "Comment both"
                ],
                label="Commenting Options",
                value="Comment input code",
                interactive=True
            )
        with gr.Row():
            in_comment_style = gr.Dropdown(choices=in_comment_style_choices, label="Select comment style for input code", value=in_comment_style_choices[0], interactive=True)
            out_comment_style = gr.Dropdown(choices=out_comment_style_choices, label="Select comment style for oupt code", value=out_comment_style_choices[0], interactive=True)
        with gr.Row():
            comment_model = gr.Dropdown(["Claude", "GPT", "Gemini"], label="Select model", value="Claude")
        with gr.Row():
            generate_comments = gr.Button("Generate comments")
            download_comments = gr.Button("Download comments")
        with gr.Row():
            in_comments = gr.Textbox(label="Comments for Input Code:", lines=10)
            out_comments = gr.Textbox(label="Comments for Output Code:", lines=10)
    
    with unit_test_section:
        # Unit test part
        with gr.Row():
            unit_test_option = gr.Radio(
                choices=[
                    "Add unit test for input code",
                    "Add unit test for output code",
                    "Add unit test for both"
                ],
                label="Unit Test Options",
                value="Add unit test for input code",
                interactive=True
            )
        with gr.Row():
            in_unit_test_style = gr.Dropdown(choices=in_unit_test_style_choices, label="Select unit test style for input code", value=in_unit_test_style_choices[0], interactive=True)
            out_unit_test_style = gr.Dropdown(choices=out_unit_test_style_choices, label="Select unit test style for oupt code", value=out_unit_test_style_choices[0], interactive=True)
        with gr.Row():
            unit_test_model = gr.Dropdown(["Claude", "GPT", "Gemini"], label="Select model", value="Claude")
        with gr.Row():
            generate_unit_test = gr.Button("Generate unit test")
            download_unit_test = gr.Button("Download unit text")
        with gr.Row():
            in_unit_test = gr.Textbox(label="Unit Test for Input Code:", lines=10)
            out_unit_test = gr.Textbox(label="Unit Test for Output Code:", lines=10)

    in_lang.change(fn=change_lang, inputs=in_lang, outputs=[in_comment_style, in_unit_test_style])
    out_lang.change(fn=change_lang, inputs=out_lang, outputs=[out_comment_style, out_unit_test_style])
    input_file.change(fn=load_file_content, inputs=input_file, outputs=in_code)
    
    convert.click(optimize, inputs=[in_lang, out_lang, in_code, input_instruct, convert_model], outputs=[out_code])
    download_code.click(download_fn, inputs=[in_code, out_code, in_code_file_name, out_code_file_name])
    
    comment_show_up.click(fn=show_comment, inputs=current_selected, outputs=[comment_section, unit_test_section, comment_show_up, unit_test_show_up, current_selected])
    unit_test_show_up.click(fn=show_unit_test, inputs=current_selected, outputs=[comment_section, unit_test_section, comment_show_up, unit_test_show_up, current_selected])

    generate_comments.click(generate_comments_fn, inputs=[comment_option, in_lang, out_lang, in_code, out_code, in_comment_style, out_comment_style, comment_model], outputs=[in_comments, out_comments])
    download_comments.click(download_fn, inputs=[in_comments, out_comments, in_comments_file_name, out_comments_file_name])
    generate_unit_test.click(generate_unit_test_fn, inputs=[unit_test_option, in_lang, out_lang, in_code, out_code, in_unit_test_style, out_unit_test_style, unit_test_model], outputs=[in_unit_test, out_unit_test])
    download_unit_test.click(download_fn, inputs=[in_unit_test, out_unit_test, in_unit_test_file_name, out_unit_test_file_name])
    
ui.launch()